NL2SQL Assistant

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd /content/drive/My Drive/

Mounted at /content/drive
/content/drive/My Drive


In [ ]:
!git clone --recurse-submodules https://github.com/HKUNLP/UnifiedSKG.git

fatal: destination path 'UnifiedSKG' already exists and is not an empty directory.


In [ ]:
%cd UnifiedSKG

/content/drive/My Drive/UnifiedSKG


In [ ]:
import sys
sys.path.append('/content/drive/My Drive/UnifiedSKG')

In [ ]:
!pip install googletrans==3.1.0a0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 3.4 MB/s 
     |████████████████████████████████| 42 kB 1.0 MB/s 
     |████████████████████████████████| 1.4 MB 52.2 MB/s 
     |████████████████████████████████| 53 kB 2.0 MB/s 
     |████████████████████████████████| 65 kB 4.1 MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16370 sha256=6a4ece59814786e00bf15098b1503bd626b9312b71d422920c70c5a30e2bce2e
  Stored in directory: /root/.cache/pip/wheels/0c/be/fe/93a6a40ffe386e16089e44dad9018ebab9dc4cb9eb7eab65ae
Successfully built googletrans


In [ ]:
!pip install transformers==4.9.2
!pip install nltk
!pip install datasets==1.14.0
!pip install sentencepiece
!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.6 MB 30.2 MB/s 
     |████████████████████████████████| 880 kB 27.8 MB/s 
     |████████████████████████████████| 3.3 MB 54.6 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895259 sha256=22641f01275c08d08fde0a2c36fa8fb66b2c56bf9a8ec9e68e3a0bf2c4deea2e
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 290 kB 28.8 MB/s 
     |████████████████████████████████| 56 kB 5.4 MB/s 
     |████████████████████████████████| 115 kB 68.0 MB/s 
     |████████████████████████████████| 212 kB 53.2 MB/s 
  

In [ ]:
import os
import time
import torch
import datasets
from googletrans import Translator 
from transformers import (
    HfArgumentParser,
    set_seed,
    AutoTokenizer
)
from utils.configue import Configure
from utils.training_arguments import WrappedSeq2SeqTrainingArguments

from filelock import FileLock
import nltk
with FileLock(".lock") as lock:
    nltk.download("punkt", quiet=True)
    nltk.download("stopwords", quiet=True)

In [ ]:
sys.argv = ['/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py', 
            '--cfg', 'Salesforce/T5_base_prefix_spider_with_cell_value.cfg', 
            '--output_dir', './tmp']
parser = HfArgumentParser((WrappedSeq2SeqTrainingArguments,))
training_args, = parser.parse_args_into_dataclasses()
set_seed(training_args.seed)
args = Configure.Get(training_args.cfg)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("hkunlp/from_all_T5_base_prefix_spider_with_cell_value2", use_fast=False)
from models.unified.prefixtuning import Model
model = Model(args)
model.load("hkunlp/from_all_T5_base_prefix_spider_with_cell_value2")

Downloading:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

prefix-tuning sequence length is 10.


Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.01G [00:00<?, ?B/s]

In [ ]:
translator = Translator()
inpt = input("Введите запрос на естественном языке:\n")
question = translator.translate(inpt, dest='en')
print(question.text)

Введите запрос на естественном языке:
Получить данные о номенклатуре за 21 января 2020 год
Get item data for January 21, 2020


In [ ]:
#struct_in = "| concert_singer | stadium : stadium_id , location , name , capacity , highest , lowest , average | singer : singer_id , name , country ( France ) , song_name , song_release_year , age , is_male | concert : concert_id , concert_name , theme , stadium_id , year | singer_in_concert : concert_id , singer_id"
text_in = question.text

In [ ]:
translator = Translator()
print("=====Запрос=====")
print(inpt)
tokenized_txt = tokenizer([text_in], max_length=1024, padding="max_length", truncation=True)
pred = tokenizer.batch_decode(
    model.generate(
      torch.LongTensor(tokenized_txt.data['input_ids']),
      torch.LongTensor(tokenized_txt.data['attention_mask']),
      num_beams=1, 
      max_length=256
      ), 
    skip_special_tokens=True 
) 
print("=====Ответ=====")
print(*pred)

=====Запрос=====
Получить данные о номенклатуре за 21 января 2020 год
=====Ответ=====
select date from 2020


In [ ]:
!pip3 install pytelegrambotapi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 213 kB 7.6 MB/s 
  Created wheel for pytelegrambotapi: filename=pyTelegramBotAPI-4.7.1-py3-none-any.whl size=196989 sha256=2a7cba39712555f58624e251774e81a87d1efa488c8bcad92bc16a9fed4a5bfe
  Stored in directory: /root/.cache/pip/wheels/84/72/5f/8a7b3ca1ef1250002cf3bac48e0c72c5ee5c9596a5b1295dc1
Successfully built pytelegrambotapi


In [ ]:
import telebot

bot = telebot.TeleBot('5420372192:AAExGe9r0icqAXTGYUSjBpKJjzUMPZZFiWg')

@bot.message_handler(commands=["start"])

def start(m, res=False):

    bot.send_message(m.chat.id, 'NL2SQL Assistant запущен!')
    bot.send_message(m.chat.id, 'Напишите ваш запрос на естественном языке:')

def nlsql(inpt):
  translator = Translator()
  question = translator.translate(inpt, dest='en')
  text_in = question.text
  translator = Translator()

  tokenized_txt = tokenizer([text_in], max_length=1024, padding="max_length", truncation=True)
  pred = tokenizer.batch_decode(
      model.generate(
        torch.LongTensor(tokenized_txt.data['input_ids']),
        torch.LongTensor(tokenized_txt.data['attention_mask']),
        num_beams=1, 
        max_length=256
        ), 
      skip_special_tokens=True 
  ) 
  
  return str(*pred)

# Получение сообщений от пользователя

@bot.message_handler(content_types=["text"])

def handle_text(message):

    bot.send_message(message.chat.id, 'Ваш запрос:\n' + message.text + '\n' + '\nПеревод запроса в SQL query:\n' + nlsql(message.text))


bot.polling(none_stop=True, interval=0)